In [27]:
# const results = [
#     ['Url', 'Anchor Text', 'External']
# ];
# var urls = document.getElementsByTagName('a');
# for (urlIndex in urls) {
#     const url = urls[urlIndex]
#     const externalLink = url.host !== window.location.host
#     if(url.href && url.href.indexOf('://')!==-1) results.push([url.href, url.text, externalLink]) // url.rel
# }
# const csvContent = results.map((line)=>{
#     return line.map((cell)=>{
#         if(typeof(cell)==='boolean') return cell ? 'TRUE': 'FALSE'
#         if(!cell) return ''
#         let value = cell.replace(/[\f\n\v]*\n\s*/g, "\n").replace(/[\t\f ]+/g, ' ');
#         value = value.replace(/\t/g, ' ').trim();
#         return `"${value}"`
#     }).join('\t')
# }).join("\n");
# console.log(csvContent)

In [21]:
def get_pdfs(links_file):
    links = open(links_file,"r",encoding='utf-8').readlines()
    files = [doc for doc in [link.split("\t")[0].replace('"',' ')[1:-1] for link in links] if doc[-4:] == ".pdf" ][:-1]
    link_to_name = {link.split("\t")[0].replace('"',' ')[1:-1]:link.split("\t")[1].replace('"',' ')[1:-1] for link in links }
    import requests
    import os
    parent_path = os.path.dirname(links_file)
    os.mkdir(parent_path+"/pdfs")
    for i,URL in enumerate(files,start=1):
        response = requests.get(URL)
        open(parent_path+"/pdfs/"+link_to_name[URL]+".pdf", "wb").write(response.content)
def get_docx(links_file):
    import os
    links = open(links_file,"r",encoding='utf-8').readlines()
    files = [doc for doc in [link.split("\t")[0].replace('"',' ')[1:-1] for link in links] if doc[-5:] == ".docx" or doc[-4:] == ".doc"][:-1]
    link_to_name = {link.split("\t")[0].replace('"',' ')[1:-1]:link.split("\t")[1].replace('"',' ')[1:-1] for link in links }
    import requests
    parent_path = os.path.dirname(links_file)
    os.mkdir(parent_path+"/docx")
    for i,URL in enumerate(files,start=1):
        response = requests.get(URL)
        open(parent_path+"/docx/"+link_to_name[URL], "wb").write(response.content)
def get_doc(links_file):
    import os
    links = open(links_file,"r",encoding='utf-8').readlines()
    files = [doc for doc in [link.split("\t")[0].replace('"',' ')[1:-1] for link in links] if doc[-4:] == ".doc"][:-1]
    link_to_name = {link.split("\t")[0].replace('"',' ')[1:-1]:link.split("\t")[1].replace('"',' ')[1:-1] for link in links }
    import requests
    parent_path = os.path.dirname(links_file)
    os.mkdir(parent_path+"/doc")
    for i,URL in enumerate(files,start=1):
        response = requests.get(URL)
        open(parent_path+"/doc/"+link_to_name[URL]+".doc", "wb").write(response.content)
for i in [16,17,18 ,19,21,22]:
    get_pdfs(str(i)+"/links")
    get_docx(str(i)+"/links")
    get_doc(str(i)+"/links")

In [16]:

for i,doc in enumerate(docs,start=1):
    doc = w.Documents.Open(os.getcwd()+"/"+doc)
    doc.SaveAs(os.getcwd()+"/5778_Docx/"+str(i)+"_.docx", 16)


In [30]:
import pandas as pd
from docx.api import Document
document = Document('2_.docx')
table = document.tables[0]
data = []

keys = None
for i, row in enumerate(table.rows):
    text = (cell.text for cell in row.cells)

    if i == 0:
        keys = tuple(text)
        continue
    row_data = dict(zip(keys, text))
    data.append(row_data)
    print (data)

df = pd.DataFrame(data)
df.to_csv("test.csv")

[{"המעורבים במקרה נשוא התלונה – שיוך (סטודנט, איש סגל וכו') \nללא פרטים מזהים": 'סטודנט לא מזוהה', 'מהות התלונה ופירוט המקרה – ללא פרטים מזהים': 'סטודנטית דיברה בטלפון והסטודנט עמד מאחוריה והתערב בשיחה תוך כדי אמירה מטרידה ובוטה.\nועלה במדרגות בלי שהיא הספיקה לראות מי הוא.\nהסטודנטית, שהיתה בהריון, החלה להרגיש רע בעקבות המקרה וחזרה הביתה.', 'מועד הגשת התלונה': 'אפריל 2018', 'מועד סיום הטיפול': 'מאי 2018', 'אופן הטיפול לרבות פירוט צעדי הביניים שננקטו וכן אמצעים נוספים שננקטו – כגון הגנה על המתלונן, ככל שננקטו': 'האחראית על ההטרדות ניסתה לאתר ולזהות את הסטודנט.\nהסטודנטית דיווחה שהיה סטודנט נוסף שעמד כעד ליד המדרגות.\n\nהסטודנט העד נקרא לשתי שיחות בירור אך הוא חדש במכללה ולא זיהה את הסטודנט המטריד.\n\nהסטודנטית המוטרדת קיבלה דיווח על כל צעד שנעשה בנדון.\n\nבעקבות המקרה, נשיא המכללה והאחראית על מניעת ההטרדות עברו בכל הכיתות, הקריאו את הסעיפים הרלוונטיים בלשון החוק והזהירו על הרחקה לצמיתות במקרה דומה.', 'המלצת האחראי': '', 'החלטת המוסד': '', 'החלטת הגוף המשמעתי/\nעונש, ככל שננקטו הליכי משמ

In [20]:
import pandas as pd
import io
import csv
from docx import Document

def read_docx_tables(filename, tab_id=None, **kwargs):
    """
    parse table(s) from a Word Document (.docx) into Pandas DataFrame(s)

    Parameters:
        filename:   file name of a Word Document

        tab_id:     parse a single table with the index: [tab_id] (counting from 0).
                    When [None] - return a list of DataFrames (parse all tables)

        kwargs:     arguments to pass to `pd.read_csv()` function

    Return: a single DataFrame if tab_id != None or a list of DataFrames otherwise
    """
    def read_docx_tab(tab, **kwargs):
        vf = io.StringIO()
        writer = csv.writer(vf)
        for row in tab.rows:
            writer.writerow(cell.text for cell in row.cells)
        vf.seek(0)
        return pd.read_csv(vf, **kwargs)

    doc = Document(filename)
    if tab_id is None:
        return [read_docx_tab(tab, **kwargs) for tab in doc.tables]
    else:
        try:
            return read_docx_tab(doc.tables[tab_id], **kwargs)
        except IndexError:
            print('Error: specified [tab_id]: {}  does not exist.'.format(tab_id))
            raise
a = read_docx_tables('18/docx/בצלאל - דיווח הטרדות מיניות')

ModuleNotFoundError: No module named 'pandas'

In [44]:
a[1]

IndexError: list index out of range

In [45]:
a[0].keys()

Index(['המעורבים במקרה נשוא התלונה – שיוך (סטודנט, איש סגל וכו') \nללא פרטים מזהים',
       'מהות התלונה ופירוט המקרה – ללא פרטים מזהים', 'מועד הגשת התלונה',
       'מועד סיום הטיפול',
       'אופן הטיפול לרבות פירוט צעדי הביניים שננקטו וכן אמצעים נוספים שננקטו – כגון הגנה על המתלונן, ככל שננקטו',
       'המלצת האחראי', 'החלטת המוסד',
       'החלטת הגוף המשמעתי/\nעונש, ככל שננקטו הליכי משמעת'],
      dtype='object')